In [2]:
#import networkx as nx
import csv
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib;
import pandas as pd 
import utm
import matplotlib.pyplot as plt 
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
from ipyleaflet import Map, FullScreenControl, Polyline
from datetime import datetime, timedelta
import subprocess
ox.config(use_cache=True, log_console=True)
%matplotlib inline
from shapely.geometry import Point
import pyproj, math


R = 6378137
f_inv = 298.257224
f = 1.0 / f_inv
e2 = 1 - (1 - f) * (1 - f)

# Sample (Lat, Lng, Alt)
ex_LLA = [
    (0,  45,  1000),
    (45, 90,  2000),
    (48.8567,  2.3508,  80),
    (61.4140105652, 23.7281341313, 149.821),
    (51.760597, -1.261247, 114.284188),
]

def gps2ecef_pyproj(lat, lon, alt):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, lon, lat, alt, radians=False)
    return x, y, z

def gps2ecef_custom(latitude, longitude, altitude):
    # (lat, lon) in WSG-84 degrees
    # altitude in meters
    cosLat = math.cos(latitude * math.pi / 180)
    sinLat = math.sin(latitude * math.pi / 180)

    cosLong = math.cos(longitude * math.pi / 180)
    sinLong = math.sin(longitude * math.pi / 180)
    f_inv = 298.257224
    f = 1.0 / f_inv
    c = 1 / math.sqrt(cosLat * cosLat + (1 - f) * (1 - f) * sinLat * sinLat)
    s = (1 - f) * (1 - f) * c

    x = (R*c + altitude) * cosLat * cosLong
    y = (R*c + altitude) * cosLat * sinLong
    z = (R*s + altitude) * sinLat

    return x, y, z

def ecef_to_enu(x, y, z, latRef, longRef, altRef):
    cosLatRef = math.cos(latRef * math.pi / 180)
    sinLatRef = math.sin(latRef * math.pi / 180)

    cosLongRef = math.cos(longRef * math.pi / 180)
    sinLongRef = math.sin(longRef * math.pi / 180)
    f_inv = 298.257224
    f = 1.0 / f_inv
    cRef = 1 / math.sqrt(cosLatRef * cosLatRef + (1 - f) * (1 - f) * sinLatRef * sinLatRef)

    x0 = (R*cRef + altRef) * cosLatRef * cosLongRef
    y0 = (R*cRef + altRef) * cosLatRef * sinLongRef
    z0 = (R*cRef*(1-e2) + altRef) * sinLatRef

    xEast = (-(x-x0) * sinLongRef) + ((y-y0)*cosLongRef)

    yNorth = (-cosLongRef*sinLatRef*(x-x0)) - (sinLatRef*sinLongRef*(y-y0)) + (cosLatRef*(z-z0))

    zUp = (cosLatRef*cosLongRef*(x-x0)) + (cosLatRef*sinLongRef*(y-y0)) + (sinLatRef*(z-z0))

    return xEast, yNorth, zUp

# Geodetic Coordinates (Latitude, Longitude, Altitude)
def geodetic_to_enu(lat, lon, h, lat_ref, lon_ref, h_ref):
    x, y, z = gps2ecef_custom(lat, lon, h)
    return ecef_to_enu(x, y, z, lat_ref, lon_ref, h_ref)

def geodetic_to_enu2(qu_LLA, rf_LLA):
    ECEF    = gps2ecef_custom(qu_LLA)
    ENU     = ecef_to_enu(ECEF, rf_LLA)
    return ENU 

with open("../dataset/gps_solution_TST/groundTruth_TST.csv", 'r') as f:
    reader = csv.reader(f, delimiter=",",quoting=csv.QUOTE_NONNUMERIC);
    data_gt_raw = list(reader);
    
# Save ground truth using UTM conversion
with open("./results/gt_utm.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for item in data_gt_raw:
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        utm_res = utm.from_latlon(item[2], item[3]) # return easting, northing, zone
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, utm_res[0], utm_res[1], item[4], 0, 0, 0, 1))
        
# Save ground truth using ECEF conversion
with open("./results/gt_ecef2.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for item in data_gt_raw:
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        ecef_x, ecef_y, ecef_z = gps2ecef_custom(item[2], item[3], item[4])
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, ecef_x, ecef_y, ecef_z, 0, 0, 0, 1))
        
# Save ground truth using ENU conversion
with open("./results/gt_enu2.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    ref_lon  =  114.179000972
    ref_lat  =  22.3011535667        
    ref_alt  =  6.4282151209
    for item in data_gt_raw:
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        enu2 = geodetic_to_enu(item[2], item[3], item[4], ref_lat, ref_lon, ref_alt)
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, enu2[0], enu2[1], enu2[2], 0, 0, 0, 1))

# EPSG:4978-ECEF, EPSG:4326-Lat,Lon , EPSG:32650-UTM 50N (Hong Kong)
# Save ground truth using ECEF conversion
data_gt_lat = [item[2] for item in data_gt_raw]
data_gt_lon = [item[3] for item in data_gt_raw]
data_gt_alt = [item[4] for item in data_gt_raw]
data_gt_gpd = gpd.points_from_xy(data_gt_lon, data_gt_lat, z=data_gt_alt, crs="EPSG:4326")
data_gt_gpd_conv = data_gt_gpd.to_crs(crs="EPSG:4978")
#print(data_gt_gpd[:10].x)                
with open("./results/gt_ecef.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_gt_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_gt_gpd_conv[i].x, data_gt_gpd_conv[i].y, data_gt_gpd_conv[i].z, 0, 0, 0, 1))

        
# Call evo_ape evaluation
f = open("./results/evo_ape.txt", "w")
subprocess.call("evo_ape tum gt_utm.tum gt_ecef.tum -va --save_plot ape.pdf", stdout=f, shell=True, cwd="./results")
f.close()
print("Finished")

def run_test():
    for pt in ex_LLA:
        xPy,yPy,zPy = gps2ecef_pyproj(pt[0], pt[1], pt[2])   
        xF,yF,zF        = gps2ecef_custom(pt[0], pt[1], pt[2])
        xE, yN, zU  = ecef_to_enu(xF,yF,zF, pt[0], pt[1], pt[2])
    
        print('\n>>> LLA: {}'.format(pt))
        print(">> pyproj (XYZ)\t = ", xPy, yPy, zPy)
        print(">> ECEF (XYZ)\t = ", xF, yF, zF)
        print('>> ENU (XYZ) \t = ', xE, yN, zU)
        print('-'*100)

#run_test()

Finished
